In [85]:
import pandas as pd
import numpy as np
import random
import gc
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [40]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

In [41]:
X, y = build_training_data(0, 243, True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X = None
y = None
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

In [42]:
class_weight = {
    0: 1,
    1: 2,
    2: 4,
    3: 8,
    4: 4,
    5: 4,
    6: 6
}

In [87]:
def train_and_test(beat_feature_model):
    print ('Train Error')
    print (beat_feature_model.evaluate(X_train, y_train, batch_size=64))
    print ('Test Error')
    print (beat_feature_model.evaluate(X_test, y_test, batch_size=64))
    print ('\n')

In [88]:
tanh
tanh
softmax
optimizer='adadelta'

Train Error
481012/481012 [==============================] - 39s    
[0.42592726305637552, 0.8627934438250856]
Test Error
120253/120253 [==============================] - 9s     
[0.42657310720665875, 0.86160844219953914]




99863

In [92]:
relu
tanh
softmax
optimizer='adam'

Train Error
480960/481012 [============================>.] - ETA: 0s[0.42942038129348753, 0.86228202206765714]
Test Error
120192/120253 [============================>.] - ETA: 0s[0.42997556868884673, 0.86154191580312245]




53825

In [89]:
tanh
tanh
softmax
sgd = SGD(lr=0.01, clipnorm=1.)
optimizer=sgd

Train Error
480896/481012 [============================>.] - ETA: 0s[0.56995858570943969, 0.84818466067325049]
Test Error
120253/120253 [==============================] - 7s     
[0.57250742078968464, 0.84692273789229022]




54172

In [91]:
relu
tanh
softmax
optimizer='adagrad'

Train Error
480960/481012 [============================>.] - ETA: 0s[0.5320718311445054, 0.84044472903062051]
Test Error
120064/120253 [============================>.] - ETA: 0s[0.53174864932115651, 0.84124304591054522]




44178

In [90]:
beat_feature_model = Sequential()

beat_feature_model.add(Dense(324, input_dim=324, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('relu'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(162, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('sigmoid'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(num_classes, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('softmax'))

sgd = RMSprop(lr=0.01)

beat_feature_model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])

beat_feature_model.fit(X_train, y_train, nb_epoch=3, batch_size=64, class_weight=class_weight, verbose = 0)
beat_feature_model.save('models/song_class_model_2.h5')
train_and_test(beat_feature_model)
beat_feature_model = None
gc.collect()

Train Error
480960/481012 [============================>.] - ETA: 0s[0.42432194990426675, 0.87033379624872576]
Test Error
120064/120253 [============================>.] - ETA: 0s[0.42543680474911666, 0.86933382120257197]




48761

In [100]:
model = Sequential()
model.add(Dense(256, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, nb_epoch=3, verbose=1, validation_data=(X_test, y_test))
train_and_test(model)
model.save('models/song_class_model_6.h5')
model = None
gc.collect()

Train on 481012 samples, validate on 120253 samples
Epoch 1/3
481012/481012 [==============================] - 66s - loss: 0.5722 - acc: 0.8493 - val_loss: 0.4487 - val_acc: 0.8574
Epoch 2/3
481012/481012 [==============================] - 70s - loss: 0.4131 - acc: 0.8699 - val_loss: 0.4260 - val_acc: 0.8654
Epoch 3/3
481012/481012 [==============================] - 68s - loss: 0.4086 - acc: 0.8717 - val_loss: 0.4108 - val_acc: 0.8709
Train Error
481012/481012 [==============================] - 26s    
[0.40830669709949413, 0.87232750949981119]
Test Error
120253/120253 [==============================] - 6s     
[0.41075646843757396, 0.87091382336468015]




50680

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=5, batch_size=32, verbose=1, validation_data=(X_test, y_test))
train_and_test(model)
model.save('models/song_class_model_7.h5')
model = None
gc.collect()


Train on 481012 samples, validate on 120253 samples
Epoch 1/5
392544/481012 [=======================>......] - ETA: 12s - loss: 0.7732 - acc: 0.8501

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=5, batch_size=32, verbose=1, validation_data=(X_test, y_test))
train_and_test(model)
model.save('models/song_class_model_7.h5')
model = None
gc.collect()
